<a href="https://colab.research.google.com/github/NgoMinhToan/colab_massive_data/blob/main/mid_term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 22.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=73ce46dcc1579be0c868757a5d1f8bfc15693a9931f0a9fd943d9f4b97d071c0
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
import re
import time
import random
from itertools import combinations
import requests
from bs4 import BeautifulSoup
import pyspark
import collections

In [4]:
def get_signgles(doc, k, signgles = set()):
    for i in range(0, len(doc) -k):
        signgles.add(doc[i:i+k])
    return signgles

In [3]:
def get_boolean_value(signgles, doc):
    boolean_value = []
    for i in signgles:
        if i in doc:
            boolean_value.append(1)
        else:
            boolean_value.append(0)
    return boolean_value

In [5]:
def Jaccard_similarity(a, b):
    total = 0
    index = 0
    for i in range(len(a)):
        if a[i] == 1 or b[i] == 1:
            total += 1
        if a[i] == b[i] == 1:
            index += 1
    if total == 0:
        return 0
    return index/total

In [6]:
def Probility_Sig(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count += 1
    return count/len(a)

In [7]:
# create hash function
def hash_function(x, a, b, c):
    return (a*x + b) % c

In [8]:
def find_min_prime(x):
    for i in range(length_shingle, length_shingle + 100000):
        prime = True
        for j in range(2, i//2):
            if i%j==0:
                prime = False
                break
        if prime:
            return i

In [9]:
k = 9
num_for_hash = 100
threshold = .5
b = 1
time_no_pyspark = 0
time_using_pyspark = 0

#**Crawl dữ liệu**

In [10]:
artical1 = []

page = "https://tuoitre.vn/thoi-su.htm"
response = requests.get(page)
soup = BeautifulSoup(response.content, "html.parser")
titles = soup.findAll('h3', class_='title-news')
links = [link.find('a').attrs["href"] for link in titles]

for link in links:
    full_link = "https://tuoitre.vn" + link
    news = requests.get(full_link)
    soup = BeautifulSoup(news.content, "html.parser")
    title = ''
    image = None
    content = ''
    abstract = ''
    if soup.find("h1", class_="article-title"):
        title = soup.find("h1", class_="article-title").text
    if soup.find("h2", class_="sapo"):
        abstract = soup.find("h2", class_="sapo").text
    body = soup.find("div", id="main-detail-body")
    if body:
        if body.find('p'):
            content = '\n'.join([i.text for i in body.findChildren("p", recursive=False)])
        if body.find("img"):
            image = body.find("img").attrs["src"]
    artical1.append({'title': title, 'abstract': abstract, 'body': body, 'content': content, 'image': image, 'link': full_link, 'homePage': 'tuoitre.vn'})


In [11]:
artical2 = []

page = "https://vnexpress.net/thoi-su"
response = requests.get(page)
soup = BeautifulSoup(response.content, "html.parser")
titles = soup.findAll('h3', class_='title-news')
links = [link.find('a').attrs["href"] for link in titles]

for link in links:
    news = requests.get(link)
    soup = BeautifulSoup(news.content, "html.parser")
    title = ''
    content = ''
    image = None
    abstract = ''
    if soup.find("h1", class_="title-detail"):
        title = soup.find("h1", class_="title-detail").text
    if soup.find("p", class_="description"):
        abstract = soup.find("p", class_="description").text
    body = soup.find("article", class_="fck_detail")
    if body:
        if body.find('p'):
            content = '\n'.join([i.text for i in body.findChildren("p", recursive=True)])
        if body.find("img"):
            image = body.find("img").attrs["src"]
    artical2.append({'title': title, 'abstract': abstract, 'body': body, 'content': content, 'image': image, 'link': link, 'homePage': 'vnexpress.net'})

In [12]:
artical3 = []

page = "https://vietnamnet.vn/vn/thoi-su/"
response = requests.get(page)
soup = BeautifulSoup(response.content, "html.parser")
titles = soup.findAll('h3', class_='box-subcate-style4-title')
links = [link.find('a').attrs["href"] for link in titles]

for link in links:
    full_link = "https://vietnamnet.vn" + link
    news = requests.get(full_link)
    soup = BeautifulSoup(news.content, "html.parser")
    title = ''
    abstract = ''
    content = ''
    image = None
    if soup.find("h1", class_="title"):
        title = soup.find("h1", class_="title").text
    body = soup.find("div", id="ArticleContent")
    if body:
        if body.find("div", class_="ArticleLead"):
            abstract = body.find("div", class_="ArticleLead").text
        if body.find("p"):
            content = '\n'.join([i.text for i in body.findChildren("p", recursive=False)])
        if body.find("img"):
            image = body.find("img").attrs["src"]
    artical3.append({'title': title, 'abstract': abstract, 'body': body, 'content': content, 'image': image, 'link': full_link, 'homePage': 'vietnamnet.vn'})

In [13]:
artical = artical1 + artical2 + artical3
documents = [re.sub('[\.,\(\)\n]', '', i['content'].lower()) for i in artical]
documents = list(filter(lambda f: len(f)>1000, documents))
print(len(documents))

46


In [ ]:

# stringA = 'Dịch Covid-19 làm tăng nhu cầu nhân lực ở nhiều lĩnh vực, trong đó lao động ngành IT với kinh nghiệm dày dặn có thể đạt mức lương trên 100 triệu đồng/tháng. Thị trường lao động đã trải qua những biến động mạnh vào năm 2020. Mới đây, Adecco Vietnam, đơn vị chuyên về giải pháp nhân sự phát hành Bảng hướng dẫn lương 2021. Tài liệu này cung cấp thống kê mức lương của hai thành phố lớn tại Việt Nam là Hà Nội và TP.HCM dựa theo kinh nghiệm làm việc của một số vị trí trong các ngành như nhân sự, luật, tài chính ngân hàng, công nghệ thông tin (CNTT), bán hàng và marketing, y tế, kỹ thuật và chế tạo. Các số liệu là mức lương gộp hàng tháng, chưa bao gồm các phúc lợi khác. Cụ thể, với xu hướng thúc đẩy quá trình chuyển đổi kỹ thuật số, nhu cầu tuyển dụng các vị trí liên quan đến CNTT như kỹ sư dữ liệu, kỹ sư cơ sở hạ tầng hay giám đốc công nghệ đang tăng lên đáng kể. Bên cạnh đó, kỹ sư phần mềm và kiến trúc sư phần mềm với kinh nghiệm dày dặn có thể đạt mức lương trên 100 triệu đồng/tháng. Trong lĩnh vực chăm sóc sức khỏe, các doanh nghiệp có xu hướng tập trung vào việc nâng cao năng lực cạnh tranh của lực lượng lao động hiện có. Một số phương pháp phổ biến nhằm nâng cao hiệu suất của nhân viên bao gồm đào tạo nội bộ, đánh giá hiệu quả công việc và bổ nhiệm lại nhân sự. Do đó, các chuyên gia về phát triển nhân tài và bán hàng được săn đón với mức lương hàng tháng tương đối cao, lên đến 80 triệu đồng. Trong ngành tài chính, các vị trí được tìm kiếm hàng đầu là giám đốc đầu tư cổ phần tư nhân, giám đốc đầu tư cổ phiếu niêm yết, giám đốc ngân hàng đầu tư và trưởng bộ phận nghiên cứu cổ phiếu cho các công ty FDI; giám đốc mua bán và sáp nhập, giám đốc điều hành và giám đốc tài chính cho các doanh nghiệp được tài trợ bởi các nhà quản lý tài sản từ Nhật Bản, Hàn Quốc, Mỹ, châu Âu, Hongkong và Singapore. Nhu cầu tuyển dụng các vị trí này có thể được xem là sự dịch chuyển lao động đáng kể trên thị trường tài chính trong các năm 2020 và 2021. Một số lĩnh vực khác cũng đang mở rộng việc săn đón nhân tài bao gồm sản xuất chế tạo, công nghệ tài chính (Fintech), logistics, nông nghiệp và bán lẻ. Theo Adecco, 2020 là một năm đầy thách thức đối với nền kinh tế Việt Nam. Sự bùng phát của dịch Covid-19 đã làm gián đoạn các hoạt động thương mại và sản xuất. Ngành du lịch và hàng không là một trong những ngành bị ảnh hưởng nặng nề nhất. Dòng vốn FDI vào Việt Nam năm ngoái giảm 25%. Việc đóng cửa các doanh nghiệp và sa thải hàng loạt khiến tỷ lệ thất nghiệp ở khu vực thành thị trong quý II/2020 đạt mức cao nhất trong vòng 10 năm qua. Tuy nhiên, nhờ những nỗ lực trong việc ngăn chặn dịch bệnh, Việt Nam được xem là điểm sáng trên bản đồ kinh tế toàn cầu. Nền kinh tế tiếp tục đạt được tốc độ tăng trưởng dương và thuộc hàng cao nhất thế giới. Lĩnh vực sản xuất và xuất khẩu đang phục hồi mạnh mẽ sau những tác động ban đầu của đại dịch. Các hiệp định thương mại như: EVFTA, UKVFTA, RCEP được ký kết giúp Việt Nam đa dạng hóa thị trường xuất khẩu và đối tác thương mại. Đồng thời, ngày càng nhiều công ty tái phân bổ chuỗi sản xuất sang Việt Nam trong bối cảnh các nhà máy tại các nước khác đang đóng cửa và cuộc chiến thương mại Mỹ - Trung vẫn tiếp diễn'
# stringB = 'Sự phát triển của AI, Big Data và bảo mật đã mở ra cơ hội mới cho nhân lực về công nghệ thông tin. Lương cho các vị trí kỹ sư IT hiện cũng có thể đạt hơn 100 triệu đồng/tháng. Theo "Bảng hướng dẫn lương 2021" của công ty giải pháp nhân sự Adecco Việt Nam, do xu hướng thúc đẩy quá trình phát triển kỹ thuật số, nhu cầu tuyển dụng nhân sự công nghệ thông tin ở các vị trí kỹ sư dữ liệu, kỹ sư cơ sở hạ tầng và giám đốc công nghệ tăng lên đáng kể. Trong đó, kiến trúc sư phầm mềm/giải pháp, kỹ sư phần mềm, kỹ sư dữ liệu có thể nhận được mức lương cao nhất lần lượt 160 triệu, 120 triệu và 80 triệu đồng mỗi tháng. Giám đốc công nghệ và giám đốc công nghệ thông tin trên 5 năm kinh nghiệm có thể nhận mức lương 250-400 triệu đồng/tháng tại TP HCM, 120-250 triệu đồng/tháng ở Hà Nội. Luong ky su cong nghe tren 100 trieu dong anh 1 Sự phát triển của AI, Big Data và bảo mật đã mở ra cơ hội mới cho các kỹ sư công nghệ thông tin. Ảnh: Adecco Việt Nam. Trong nhóm ngành chăm sóc sức khỏe, các vị trí quản trị tài năng và nhân lực bán hàng hiệu quả rất được săn đón, với mức lương hàng tháng lên đến 80 triệu đồng. Trong ngành tài chính, các vị trí hàng đầu là: giám đốc đầu tư cổ phần tư nhân, giám đốc đầu tư cổ phiếu niêm yết, giám đốc ngân hàng đầu tư, trưởng bộ phận nghiên cứu cổ phiếu cho các công ty FDI, giám đốc mua bán và sáp nhập, giám đốc điều hành, giám đốc tài chính cho doanh nghiệp được tài trợ bởi các nhà quản lý tài sản từ Nhật Bản, Hàn Quốc, Mỹ, châu Âu, Hong Kong và Singapore. Nhu cầu tuyển dụng các vị trí trên đây có thể được xem là sự dịch chuyển lao động đáng kể trên thị trường tài chính trong các năm 2020 và 2021. Trong đó, vị trí giám đốc đầu tư cổ phần tư nhân từ 5 năm kinh nghiệm có thể nhận mức lương 180-250 triệu đồng mỗi tháng tại TP HCM, 150-215 triệu đồng ở Hà Nội. Một số lĩnh vực khác cũng đang mở rộng việc săn đón nhân tài bao gồm: sản xuất chế tạo, công nghệ tài chính (Fintech), quản lý chuỗi cung ứng, nông nghiệp và bán lẻ. Luong ky su cong nghe tren 100 trieu dong anh 2 Giám đốc đầu tư cổ phần tư nhân từ 5 năm kinh nghiệm có thể nhận mức lương 180-250 triệu đồng mỗi tháng tại TP HCM, 150-215 triệu đồng ở Hà Nội. Ảnh: Adecco Việt Nam. Adecco Việt Nam cũng cho hay trong năm 2020, do ảnh hưởng của đại dịch Covid-19, tăng trưởng GDP năm 2020 của Việt Nam thấp nhất trong thập kỷ (2011 - 2020), ở mức 2,91%. Nhiều hoạt động thương mại và sản xuất bị gián đoạn, dòng vốn FDI vào Việt Nam giảm 25%. Việc đóng cửa doanh nghiệp và sa thải hàng loạt khiến tỷ lệ thất nghiệp ở khu vực thành thị đạt mức cao nhất trong 10 năm vào quý 2/2020. Du lịch và hàng không là một trong những ngành bị ảnh hưởng nặng nề nhất. Tuy nhiên, nhờ thành công trong việc ngăn chặn virus, Việt Nam được coi là điểm sáng trên bản đồ kinh tế toàn cầu. Đây còn là điểm đến của các nhà đầu tư nước ngoài bao gồm Mỹ, Singapore, Hàn Quốc, Trung Quốc, Nhật Bản...Việt Nam đứng thứ 33 trong danh sách 100 thương hiệu quốc gia giá trị nhất thế giới, tăng 9 bậc so với 2019'
# stringC = 'Nhu cầu cao cùng nguồn cung khan hiếm đã khiến nghề kỹ sư IT cùng một số ngành nghề khác đang được săn đón với mức lương khiến nhiều người ngưỡng mộ. Theo VnExpress, công ty giải pháp nhân sự Adecco Việt Nam mới đây đã phát hành Bảng hướng dẫn lương 2021. Trong đó cho biết, nhu cầu tuyển dụng của một số vị trí thuộc ngành công nghệ thông tin đang có xu hướng tăng lên đáng kể. Cụ thể, 3 vị trí trong ngành này được đánh giá hot nhất là giám đốc công nghệ, kỹ sư dữ liệu và kỹ sư cơ sở hạ tầng. IT hiện là một trong những ngành được trả lương khá cao ở Việt Nam. (Ảnh: Grow Up Work) IT hiện là một trong những ngành được trả lương khá cao ở Việt Nam. (Ảnh: Grow Up Work) Với trung bình 5 năm kinh nghiệm, một người ở vị trí giám đốc công nghệ và giám đốc công nghệ thông tin có thể nhận mức lương thấp nhất là 120 triệu đồng và cao nhất lên đến 400 triệu đồng tùy khu vực. Ngoài ra các vị trí như kiến trúc sư phần mềm/giải pháp, kỹ sư phần mềm, kỹ sư dữ liệu cũng có thể nhận các mức lương tương ứng lần lượt là 160 triệu đồng, 120 triệu đồng và 80 triệu đồng mỗi tháng. Mức lương thuộc một số vị trí ngành Công nghệ thông tin hiện nay. (Ảnh: Adecco Việt Nam) Mức lương thuộc một số vị trí ngành Công nghệ thông tin hiện nay. (Ảnh: Adecco Việt Nam) Nguyên nhân của mức lương khủng này theo Adecco đánh giá là bởi quá trình chuyển đổi kỹ thuật số đang phát triển mạnh. Do đó, hiện giờ đang là kỷ nguyên bùng nổ cho các chuyên gia IT có năng lực. Sự phất lên của trí tuệ nhân tạo (AI), dữ liệu lớn (Big Data) cũng đòi hỏi một lượng lớn nhân lực có chuyên môn từ nghiệp vụ đến kỹ năng giao tiếp, khả năng ngôn ngữ. Song các ứng viên đủ điều kiện lại không nhiều nên lẽ thường tình, mức lương đưa ra cũng phải tăng lên để giữ chân người tài. Nguồn nhân sự cho ngành IT vẫn luôn cung ít hơn cầu. (Ảnh: VnExpress) Nguồn nhân sự cho ngành IT vẫn luôn cung ít hơn cầu. (Ảnh: VnExpress) Tuy nhiên theo như chia sẻ đến từ chính người trong nghề, tất cả những đánh giá này cũng chỉ là bề nổi mà thôi. Anh T.T cho biết: “Kinh nghiệm 5 năm mà có mức lương như trên thì cũng khá hiếm, phải làm việc trong công ty hàng top hoặc thực lực phải rất giỏi. Nói chung, lương lên được mức hơn trăm triệu cũng thuộc hàng “siêu nhân” rồi. Anh em bình thường đa số với từng đó năm kinh nghiệm cũng chỉ loay hoay 1/3 mức này thôi. Có làm rồi mới biết cực.” Có ý kiến lại cho rằng, với mức lương lý tưởng thế này, vô hình chung đã tạo động lực cho rất nhiều bạn trẻ mục tiêu để theo đuổi. Nhất là trong tình cảnh ngành IT chưa có dấu hiệu đủ nhân lực. Chỉ cần có năng lực, đủ đam mê thì không ngại học xong không có đầu ra. Để được hưởng mức lương cao này thì cũng phải có năng lực xứng tầm mới được. (Ảnh: VnExpress) Để được hưởng mức lương cao này thì cũng phải có năng lực xứng tầm mới được. (Ảnh: VnExpress) Đối với những cô gái không đủ sức “trâu bò” để theo đuổi công việc đặc thù này thì có thể nghĩ đến chuyện lấy một anh chồng làm nghề IT . Nói cho vui vậy chứ các cặp đôi không phải lúc nào cũng đến với nhau vì kinh tế. Chưa kể con trai làm IT đôi khi lại không thỏa mãn được tâm hồn lãng mạn của các bạn gái đâu. Mỗi ngành nghề đều có cái giá của nó và như nghề IT cũng vậy. Để có được lương cao thì đồng nghĩa với việc phải đánh đổi nhiều vì hiện tại làm gì có việc nhẹ lương cao tồn tại nữa đâu.'

# stringA = re.sub('[\.,\(\)]', '', stringA.lower())
# stringB = re.sub('[\.,\(\)]', '', stringB.lower())
# stringC = re.sub('[\.,\(\)]', '', stringC.lower())
# documents = [stringA, stringB, stringC]

--------------------------------------------------------
#**Không sử dụng PySpark**

In [14]:
t0 = time.time()

signgles = set()
for doc in documents:
    get_signgles(doc, k, signgles)
length_shingle = len(signgles)

time_no_pyspark += (time.time() - t0)

# print('The Union K-gram:', signgles)
# print()

In [15]:
t0 = time.time()

boolean_vector = []
for doc in documents:
    boolean_vector.append(get_boolean_value(signgles, doc))

time_no_pyspark += (time.time() - t0)

# print('The matrix boolean: ')
# for i in range(len(boolean_vector)):
#     print('C{}:'.format(i+1), boolean_vector[i])
# print()


In [16]:
# Function Hash
hash_func_arr = []

# tim so nguyen to nho nhat nhung lon hon x
c = find_min_prime(length_shingle)

for i in range(num_for_hash):
    hash_func_arr.append([random.randint(1, length_shingle), random.randint(1, length_shingle), c])
# print(hash_func_arr)


In [ ]:
# print('Jaccard similarity: ')
# for i1, i2 in combinations(range(len(boolean_vector)), 2):
#     print('C{} - C{}: '.format(i1+1, i2+1), Jaccard_similarity(boolean_vector[i1], boolean_vector[i2]))
# print()


In [17]:
t0 = time.time()

# Min Hash Using Function
min_hash = [[1000000000 for i in range(num_for_hash)] for j in range(len(boolean_vector))]
for r in range(length_shingle):
    hash_row = [hash_function(r, *a) for a in hash_func_arr]
    for c in range(len(boolean_vector)):
        if boolean_vector[c][r] == 1:
            for i in range(num_for_hash):
                if min_hash[c][i] > hash_row[i]:
                    min_hash[c][i] = hash_row[i]

time_no_pyspark += (time.time() - t0)

# print('Min Hash:')
# for i in range(num_for_hash):
#     print(list(map(lambda a: a[i], min_hash)))
# print()


In [18]:
print('LSH')
r = len(min_hash)//b
for i1, i2 in combinations(range(len(min_hash)), 2):
    if Probility_Sig(min_hash[i1], min_hash[i2]) > threshold:
        print('candidate: ', Probility_Sig(min_hash[i1], min_hash[i2]))
        print(min_hash[i1])
        print(min_hash[i2])
        print(artical[i1]['link'])
        print(artical[i2]['link'])
        print('--------------------------------------------')

print('Total time execute: ', time_no_pyspark)


LSH
candidate:  0.26
[45, 19, 17, 79, 81, 88, 7, 55, 64, 15, 21, 110, 0, 12, 8, 66, 45, 24, 19, 12, 57, 93, 22, 32, 128, 16, 4, 0, 108, 65, 15, 21, 48, 11, 35, 10, 32, 3, 178, 50, 15, 152, 28, 5, 26, 120, 10, 30, 8, 11, 47, 64, 6, 29, 2, 63, 11, 116, 133, 7, 44, 8, 27, 18, 102, 4, 4, 63, 31, 72, 9, 5, 0, 9, 37, 22, 39, 38, 4, 2, 47, 80, 28, 56, 4, 30, 4, 22, 37, 37, 78, 26, 22, 19, 39, 23, 8, 23, 31, 47]
[28, 67, 13, 131, 48, 242, 10, 23, 64, 15, 216, 47, 41, 12, 26, 99, 45, 129, 46, 48, 57, 109, 22, 16, 27, 6, 11, 121, 81, 65, 106, 28, 16, 11, 44, 10, 19, 3, 178, 116, 15, 37, 28, 209, 26, 165, 113, 19, 47, 11, 88, 22, 30, 29, 2, 63, 11, 130, 53, 7, 20, 32, 36, 42, 109, 57, 25, 34, 31, 62, 10, 8, 41, 214, 46, 35, 2, 39, 45, 26, 68, 24, 28, 35, 53, 109, 29, 88, 175, 63, 22, 6, 22, 165, 137, 23, 8, 23, 166, 51]
https://vnexpress.net/pho-thu-tuong-vu-duc-dam-tiem-thu-vaccine-nano-covax-4254285.html
https://vnexpress.net/thu-truong-le-minh-hoan-dan-mien-tay-dan-thich-ung-bien-doi-khi-hau-4


--------------------------------------------------------
#**Sử dụng PySpark**

In [19]:

conf = pyspark.SparkConf().setMaster('local').setAppName('Similar Document')
sc = pyspark.SparkContext.getOrCreate(conf=conf)

In [20]:
t0 = time.time()

doc = sc.parallelize(documents)
signgles_pyspark = doc.map(lambda a: get_signgles(a, k, set()))
signgles_pyspark = set(signgles_pyspark.reduce(lambda a, b: list(a) + list(b)))
length_shingle = len(signgles)

time_using_pyspark += (time.time() - t0)

# print('The Union K-gram:', signgles)
# print(length_shingle)

In [21]:
t0 = time.time()

B_vector_pyspark = doc.map(lambda d: get_boolean_value(signgles_pyspark, d))
B_vector = B_vector_pyspark.collect()

time_using_pyspark += (time.time() - t0)

# print('The matrix boolean: ')
# for i in range(len(B_vector)):
#     print('C{}:'.format(i+1), B_vector[i])
# print()

In [22]:
t0 = time.time()

min_hash_pyspark = B_vector_pyspark.map(lambda c: [min([hash_function(i, *hash_func_arr[hash_index]) for i in range(len(signgles)) if c[i] == 1] + [1000000000]) for hash_index in range(num_for_hash)])
min_hash = min_hash_pyspark.collect()

time_using_pyspark += (time.time() - t0)

# print('Min Hash:')
# for i in range(num_for_hash):
#     print(list(map(lambda a: a[i], min_hash)))
# print()


In [ ]:
print('LSH')
r = len(min_hash)//b
for i1, i2 in combinations(range(len(min_hash)), 2):
    if Probility_Sig(min_hash[i1], min_hash[i2]) > threshold:
        print('candidate: ', Probility_Sig(min_hash[i1], min_hash[i2]))
        print(min_hash[i1])
        print(min_hash[i2])
        print(artical[i1]['link'])
        print(artical[i2]['link'])
        print('--------------------------------------------')

print('Time execute: ', time_using_pyspark)
